# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install --upgrade crewai crewai_tools langchain_community

In [29]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [30]:
from crewai import Agent, Task, Crew, Process, LLM

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
# import os
# from utils import get_openai_api_key, get_serper_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [31]:
import os
from google.colab import userdata
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## crewAI Tools

In [32]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='/content/resume.md')
semantic_search_resume = MDXSearchTool(mdx='/content/resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [14]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [33]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [34]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [35]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [36]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [37]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [38]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [39]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [40]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [41]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [42]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    # 'job_posting_url': 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4147111765',
    'github_url': 'https://github.com/iammkullah',
    'personal_writeup': """ A results-driven data scientist/analyst with 2 years of experience in e-commerce, stock market analysis,
 and software development. Proven track record of leveraging advanced analytics and data technologies to
 optimize operations, drive revenue, and deliver strategic insights. Adept at using machine learning and AI
 techniques to solve complex problems and enhance business performance. Possesses strong analytical and
 problem-solving skills with a focus on data-driven decision-making. Recognized for delivering impactful so
lutions that contribute to organizational growth and strategic decision-making."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [43]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/iammkullah) URLs, and personal write-up ( A results-driven data scientist/analyst with 2 years of experience in e-commerce, stock market analysis,
 and software development. Proven track record of leveraging advanced analytics and data technologies to
 optimize operations, drive revenue, and deliver strategic insights. Adept at using machine learning and AI
 techniques to solve complex problems and enhance business performance. Possesses strong analytical and
 problem-solving skills with a focus on data-driven decision-m

ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'
ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Ser

 

I encountered an error while trying to use the tool. This was the error: 403 Client Error: Forbidden for url: https://google.serper.dev/search.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: Resume Strategist for Engineers
## Thought: Thought: Now that I have the current content of the resume, I'll proceed to incorporate the job requirements for the Full Stack Engineer position, emphasizing the must-have skills, relevant experiences, and necessary qualifications.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"best resume examples for Full Stack Engineer position\"}"
## Tool Output: 

I encountered an error w

- Dislplay the generated `tailored_resume.md` file.

In [44]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Step 1 - Below is the tailored resume content highlighting qualifications and experiences that closely match the job requirements for the Full Stack Engineer position at AI Fund. 

*Complete resume tailored for Full Stack Engineer position:*

**MUHAMMAD KALEEM ULLAH**  
[**iammkullah@gmail.com**](mailto:iammkullah@gmail.com) | [**+92 303 2225720**](tel:+92%20303%202225720)  
Islamabad, Pakistan | [iammkullah](https://linkedin.com/in/iammkullah) | [GitHub](https://github.com/iammkullah) | [Medium](https://medium.com/@iammkullah)

**ABOUT ME**  
Results-driven Full Stack Developer with over 2 years of experience in developing desktop and mobile applications. Proven track record in leveraging advanced analytics, machine learning, and AI technologies to build scalable applications and deliver impactful solutions. Strong command over front-end and back-end technologies, databases, and cloud services. A self-starter with excellent communication skills, adept at navigating fast-paced environments and meeting project deadlines.

**SKILLS**  
- **Programming Languages:** HTML, CSS, JavaScript, Python, Java, SQL, C++, C  
- **Frameworks and Libraries:** Angular, React, Node.js, jQuery  
- **Databases:** MySQL, MongoDB, Amazon DynamoDB  
- **Technologies:** Generative AI, Machine Learning, AWS (EC2, S3), GCP, Docker, Git  
- **Project Management:** Trello, ClickUp, Notion  
- **Tools:** Selenium, Scrapy, Excel, Tableau, Power BI  

**PROFESSIONAL EXPERIENCE**  

**Full Stack Developer** | Bave Technologies  
*Jan 2024 – Present | Remote, Pakistan*  
- Design and implement robust backend automation and AI solutions using Python and AWS services.
- Developed scalable desktop and mobile applications while leveraging LLMs and generative AI to analyze stock market trends, delivering actionable insights.
- Automated data enrichment processes and backend architectures, enhancing operational efficiency.

**Junior Full Stack Developer** | Acgile  
*Aug 2023 – Dec 2023 | Islamabad, Pakistan*  
- Developed effective data analysis solutions for an eCommerce platform using HTML, CSS, JavaScript, and Python.
- Created interactive dashboards and optimized web applications to improve user experience and operational efficiencies.
- Managed API integrations to synchronize inventory data across eCommerce platforms, enhancing data accuracy.

**Data Science Fellow** | Bytewise Limited  
*Mar 2023 – Jun 2023 | Remote, Pakistan*  
- Conducted advanced analyses using machine learning techniques, conducted data visualization, and supported software development projects aligning with best practices in data handling.

**Data Scientist Intern** | Technocolabs Softwares  
*Sep 2022 – Oct 2022 | Remote, India*  
- Developed and implemented machine learning models, enhancing application functionalities and user engagement.

**VOLUNTEER EXPERIENCE**  

**Team Lead** | Google Developers Group Cloud Islamabad  
*Jul 2023 – Present | Islamabad, Pakistan*  
- Led logistics and outreach for GCP-focused events, enhancing the developer ecosystem.
- Developed strong leadership and organizational skills through volunteer management roles.

**EDUCATION**  

**Bachelor of Science in Computer Engineering** | COMSATS University Islamabad Attock Campus  
*Sep 2019 – Jul 2023 | Attock, Pakistan*  
- Achieved Campus Gold Medalist with CGPA 3.79 while ranking in the top 3 throughout my educational career.
- Served as Google DSC Lead and General Secretary of IEEE and CSS, leading student initiatives.

**CERTIFICATIONS**  
- 2024 Gemini API by Google - Udacity  
- 2023 Data Analysis in Excel - DataCamp  
- 2023 Project Initiation - Google, Coursera  
- 2023 Foundations of Project Management - Google, Coursera  

**PROJECTS**  
- Developed and fine-tuned various applications utilizing cutting-edge AI technologies, including a custom resume builder and an AI-powered financial Q&A engine.
- Conducted web scraping and automated numerous data handling tasks for significant eCommerce platforms.

- Dislplay the generated `interview_materials.md` file.

In [45]:
display(Markdown("./interview_materials.md"))

1. **Experience as a Full Stack Developer**:
   - Can you describe a project where you functioned as a Full Stack Developer? What technologies did you use, and what were the outcomes?
   - Discuss your experience with both front-end and back-end development. How do you approach a full stack project?

2. **Mobile and Desktop Applications**:
   - In your role at Bave Technologies, you developed applications leveraging generative AI. Can you elaborate on that experience, especially regarding mobile vs. desktop application challenges?
   - How do you ensure that your mobile applications provide a good user experience compared to desktop applications?

3. **Familiarity with Technology Stacks**:
   - Can you discuss the technology stack you're most comfortable with, and how have you applied it in past projects?
   - In your experience, have you had to learn new technologies quickly? How did you go about that process?

4. **Front-End Languages & Libraries**:
   - You've mentioned working with HTML, CSS, and JavaScript extensively. How do you stay updated with the latest trends and best practices in front-end development?
   - Can you share a specific instance where you resolved a challenging issue in your front-end code? What tools did you use?

5. **Object-Oriented Languages**:
   - You have experience with both Python and Java. Could you compare your experiences with both languages and discuss scenarios where you preferred one over the other?
   - Share an instance where object-oriented programming significantly impacted your project's design or architecture.

6. **Frameworks Knowledge**:
   - Which frameworks (Angular, React, Node.js) do you have the most experience with, and how have you implemented them in your projects?
   - Discuss a challenging problem you solved using one of these frameworks.

7. **Database Familiarity**:
   - Explain your experience with MySQL and MongoDB. When would you choose one over the other for a project?
   - Give an example of a project where database management played a crucial role.

8. **Web Servers and UI/UX Design**:
   - Share your experience working with web servers like Apache. What challenges did you encounter, and how did you overcome them?
   - How do you incorporate UI/UX design principles in your development process?

9. **Communication Skills**:
   - Can you provide an example of a time you had to explain a technical concept to a non-technical audience? How did you ensure clarity and understanding?
   - How do you foster effective communication in a remote work environment?

10. **Self-Starter Mindset in Fast-Paced Environments**:
    - Describe a situation where you took the initiative on a project. What was the outcome, and what did you learn from that experience?
    - How do you prioritize tasks when you have multiple projects with tight deadlines?

11. **Project Management Skills**:
    - Discuss how you manage project timelines and ensure that deliverables are met. What tools (Trello, ClickUp, Notion) do you prefer and why?
    - Can you provide an example of a project where strong project management skills led to success?

12. **Nice to Haves**:
    - Do you have any experience with AI/ML models, and how have you integrated them into your applications?
    - What is your understanding of big data tools like Hadoop or Spark? Have you had the chance to work with them?

**Characteristics and Personal Qualities**:
1. **Accountability and Ownership**:
   - Can you share a challenging incident at work where you had to take accountability? What were your learning points?
   - Describe how you ensure ownership in your projects, especially regarding outcomes and timelines.

2. **Grit and Perseverance**:
   - Provide an example of a project that didn’t go as planned. How did you handle that, and what did you learn?
   - How do you maintain motivation and productivity during stressful times?

3. **Scrappiness and Resourcefulness**:
   - Can you illustrate a scenario where you had limited resources but still successfully completed a project?
   - What strategies do you use to resolve issues when encountering unexpected challenges?

Candidates should prepare elaborate answers that highlight their skills and experiences relating to the job description, using metrics and examples from previous experiences when applicable.
```